# 第一个模型微调
目标：改变模型的自我介绍
https://www.lixueduan.com/posts/ai/05-finetune-llamafactory/

## 启动 llamafactory-cli webui

In [ ]:
!export USE_MODELSCOPE_HUB=1 && export GRADIO_SERVER_PORT=6006; llamafactory-cli webui

## 准备数据集
### 将模版里面的identity.json的占位符替换

In [4]:
import json

input_file = "/root/LLaMA-Factory/data/identity.json"        # 你的原始数据文件
output_file = "/root/LLaMA-Factory/data/identity_filled.json"  # 替换后的输出文件

name = "小灵助手"
author = "SmartAI"

with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

for item in data:
    for key in ("instruction", "input", "output"):
        if key in item and isinstance(item[key], str):
            item[key] = (
                item[key]
                .replace("{{name}}", name)
                .replace("{{author}}", author)
            )

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"✅ 已生成替换后的文件: {output_file}")

✅ 已生成替换后的文件: /root/LLaMA-Factory/data/identity_filled.json


### identity_filled.json注册到/root/LLaMA-Factory/data/dataset_info.json
``` json
{
  "identity": {
    "file_name": "identity.json"
  },
  "identity_filled": {
    "file_name": "identity_filled.json"
  },
...
```

## 训练
- 选择需要微调的模型，可以是本地下载好的， 也可以是线上的， 国内建议选modelscope(魔塔社区)下载模型
- 选择lora微调
- 选择sft(监督指令微调)
- 数据集选择identity_filled 

    <img src="images/train.jpg" height="300">
- 使用swanlab记录训练过程(可选)，填入api

    <img src="images/swanlib.jpg" height="300">
- 训练过程中会实时打印训练日志，其中就包括了 loss 信息
    成功的训练一般有明显的收敛过程，且收敛出现在训练过程的后半部分都是合理的。

    1. 如果没有明显收敛，说明训练不充分，可以增加训练epoch重训，或者进行增量训练。
    2. 如果收敛出现在训练过程的前半部分，而后部分的loss平稳无变化，说明可能有过拟合，可以结合评估结果选择是否减少epoch重训。
    3. 如果有收敛趋势，但没有趋于平稳，可以在权衡通用能力和专业能力的前提下考虑是否增加epoch和数据以提升专业能力，但会有通用能力衰减的风险。
    可以看到，在训练到 70 步再往后的时候已经收敛了，看起来本次训练效果还可以

   <img src="images/training_loss.png" height="300">

##

## 合并训练模型
将训练好的lora模型和主模型合并，导出到目标路径

<img src="images/export.jpg" height="300">

## 验证效果
- 加载老模型
自我认知是QWen

<img src="images/loadOldModel.jpg" height="300">

- 加载新模型
自我认真已经改成训练数据的小灵了

------------
<img src="images/loadNewModel.jpg" height="300">

-----------
<img src="images/newModel.jpg" height="300">
